## PROBNA SVESKA

In [ ]:
# DODACEMO ARUCO NA SLIKAMA
import cv2
import numpy as np
import os

IMAGE_PATHS = []
for file in os.listdir('test_data'):
    if file.endswith('.jpg'):
        image_path = os.path.join('test_data', file)
        IMAGE_PATHS.append(image_path)

OUTPUT_IMAGE_PATH = 'test_data_aruco'
ARUCO_DICTIONARY = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
ARUCO_ID = 23
MARKER_SIZE_IN_PIXELS = 150
MARGIN_FROM_CORNER_IN_PIXELS = 25


In [ ]:
# ARUCO marker


ARUCO_DICTIONARY = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
ARUCO_ID = 23
IMAGE_SIZE_PIXELS = 80

FILENAME = "Aruco_Marker_ID_23.png"

print(f"Generišem marker sa ID={ARUCO_ID} iz rečnika DICT_4X4_50...")

# Kreiranje prazne slike (canvas)
marker_image = np.zeros((IMAGE_SIZE_PIXELS, IMAGE_SIZE_PIXELS), dtype=np.uint8)

# Crtanje Aruco markera na sliku
marker_image = cv2.aruco.generateImageMarker(
    ARUCO_DICTIONARY,
    ARUCO_ID,
    IMAGE_SIZE_PIXELS,
    marker_image,
    1 # borderBits
)

# Čuvanje slike
cv2.imwrite(FILENAME, marker_image)

print(f"Marker je uspešno sačuvan kao '{FILENAME}'")

Generišem marker sa ID=23 iz rečnika DICT_4X4_50...
Marker je uspešno sačuvan kao 'Aruco_Marker_ID_23.png'


In [ ]:

def add_aruco_to_image(input_path, output_path):
    """
    Učitava sliku, generiše Aruco marker i postavlja ga u donji levi ugao.
    """
    background_image = cv2.imread(input_path)
    if background_image is None:
        print(f"Greška: Nije moguće učitati sliku sa putanje: {input_path}")
        return

    print(f"Originalna slika učitana (Dimenzije: {background_image.shape[1]}x{background_image.shape[0]})")

    print(f"Generišem Aruco marker sa ID: {ARUCO_ID} i veličinom: {MARKER_SIZE_IN_PIXELS}x{MARKER_SIZE_IN_PIXELS} px...")
    marker_image = np.zeros((MARKER_SIZE_IN_PIXELS, MARKER_SIZE_IN_PIXELS), dtype=np.uint8)
    marker_image = cv2.aruco.generateImageMarker(
        ARUCO_DICTIONARY,
        ARUCO_ID,
        MARKER_SIZE_IN_PIXELS,
        marker_image,
        1
    )
    bgr_marker_image = cv2.cvtColor(marker_image, cv2.COLOR_GRAY2BGR)
    img_height, img_width, _ = background_image.shape

    y_start = img_height - MARGIN_FROM_CORNER_IN_PIXELS - MARKER_SIZE_IN_PIXELS
    y_end = img_height - MARGIN_FROM_CORNER_IN_PIXELS

    x_start = MARGIN_FROM_CORNER_IN_PIXELS
    x_end = MARGIN_FROM_CORNER_IN_PIXELS + MARKER_SIZE_IN_PIXELS

    if y_start < 0 or x_end > img_width:
        print("Greška: Veličina markera ili margine je prevelika za datu sliku.")
        return

    print(f"Postavljam marker na poziciju (y,x): ({y_start}, {x_start})")
    background_image[y_start:y_end, x_start:x_end] = bgr_marker_image

    cv2.imwrite(output_path, background_image)
    print(f"Nova slika je uspešno sačuvana na: {output_path}")

    cv2.imshow('Slika sa Aruco Markerom', background_image)
    print("Pritisnite bilo koje dugme da zatvorite prozor sa slikom.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
OUTPUT_IMAGE_PATH = 'test_data_aruco'
for idx, INPUT_IMAGE_PATH in enumerate(IMAGE_PATHS):
    OUTPUT_IMAGE_PATH = os.path.join(OUTPUT_IMAGE_PATH, f"image_with_aruco_{idx+1}.jpg")
    add_aruco_to_image(INPUT_IMAGE_PATH, OUTPUT_IMAGE_PATH)
    OUTPUT_IMAGE_PATH = 'test_data_aruco'

Originalna slika učitana (Dimenzije: 3000x4000)
Generišem Aruco marker sa ID: 23 i veličinom: 150x150 px...
Postavljam marker na poziciju (y,x): (3825, 25)
Nova slika je uspešno sačuvana na: test_data_aruco\image_with_aruco_1.jpg
Pritisnite bilo koje dugme da zatvorite prozor sa slikom.
Greška: Nije moguće učitati sliku sa putanje: test_data\1000026849_output.jpg
Originalna slika učitana (Dimenzije: 3000x4000)
Generišem Aruco marker sa ID: 23 i veličinom: 150x150 px...
Postavljam marker na poziciju (y,x): (3825, 25)
Nova slika je uspešno sačuvana na: test_data_aruco\image_with_aruco_3.jpg
Pritisnite bilo koje dugme da zatvorite prozor sa slikom.
Originalna slika učitana (Dimenzije: 3000x4000)
Generišem Aruco marker sa ID: 23 i veličinom: 150x150 px...
Postavljam marker na poziciju (y,x): (3825, 25)
Nova slika je uspešno sačuvana na: test_data_aruco\image_with_aruco_4.jpg
Pritisnite bilo koje dugme da zatvorite prozor sa slikom.
Originalna slika učitana (Dimenzije: 3000x4000)
Generišem 

In [ ]:
import cv2
import numpy as np
import serial
import time
import os

# Kalibracija koordinatnog sistema

def get_pixel_to_mm_ratio(image, marker_id, real_marker_size_mm):
    """
    Ponalazi Aruco marker na slici i izračunava odnos piksela i milimetara.
    Za jednostavnost, ova funkcija pretpostavlja da je kamera direktno iznad
    i vraća prosečnu širinu markera u pikselima po milimetru.

    Args:
        image: Ulazna slika.
        marker_id: ID Aruco markera koji se traži.
        real_marker_size_mm: Stvarna veličina stranice markera u mm.

    Returns:
        Odnos piksela po milimetru ili None ako marker nije pronađen.
    """
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

    corners, ids, rejected = detector.detectMarkers(image)

    if ids is not None:
        for i, current_id in enumerate(ids):
            if current_id == marker_id:
                marker_corners = corners[i][0]

                width_px = np.linalg.norm(marker_corners[0] - marker_corners[1])
                height_px = np.linalg.norm(marker_corners[1] - marker_corners[2])

                avg_size_px = (width_px + height_px) / 2.0

                pixel_per_mm = avg_size_px / real_marker_size_mm
                print(f"Marker pronađen. Odnos: {pixel_per_mm:.2f} piksela/mm")
                return pixel_per_mm

    print("Aruco marker nije pronađen na slici.")
    return None

# Segmentacija

def separate_connected_objects(mask, original_image_bgr):
    """
    Prima masku sa spojenim objektima i pokušava da ih razdvoji
    koristeći eroziju za pronalaženje semena i Watershed algoritam.
    """
    # 1. Jaka erozija da se prekinu veze i pronađu "jezgra" (semena)
    # Veličina kernela je ključna! Možda će biti potrebno podešavanje.
    kernel_erode = np.ones((19, 19), np.uint8)
    eroded_for_seeds = cv2.erode(mask, kernel_erode, iterations=2) # Dve iteracije za jači efekat

    # 2. Pronalaženje semena na erodiranoj masci
    num_labels, markers = cv2.connectedComponents(eroded_for_seeds)

    # Ako nismo dobili više od jednog objekta, razdvajanje nije uspelo.
    # num_labels uključuje i pozadinu (labela 0), tako da tražimo više od 2.
    if num_labels < 3:
        # Nema dovoljno semena, vraćamo originalne markere za jedan objekat
        print("Upozorenje: Razdvajanje nije uspelo, objekat se tretira kao jedan.")
        _, markers = cv2.connectedComponents(mask)
        return markers

    print(f"Pronađeno {num_labels - 1} semena, pokušavam razdvajanje...")

    # 3. Priprema markera za Watershed
    # Dodajemo 1 na sve labele da bi pozadina bila 0
    markers = markers + 1

    # 4. Primena Watershed algoritma
    # Algoritam radi na BGR slici, pa moramo konvertovati našu masku
    mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    cv2.watershed(mask_bgr, markers)

    # Vraćamo markere. Granice će imati vrednost -1.
    return markers


# --- AŽURIRANA `segment_objects` FUNKCIJA ---
def segment_objects(image):
    # Koraci 1, 2, 3 i 4 ostaju isti kao u verziji 6.0
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_wood = np.array([10, 100, 120])
    upper_wood = np.array([25, 255, 255])
    mask = cv2.inRange(hsv_image, lower_wood, upper_wood)
    kernel_close = np.ones((21, 21), np.uint8)
    closed_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_close)
    contours, _ = cv2.findContours(closed_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    final_mask = np.zeros(closed_mask.shape, dtype=np.uint8)
    min_area_threshold = 1000
    for cnt in contours:
        if cv2.contourArea(cnt) > min_area_threshold:
            cv2.drawContours(final_mask, [cnt], -1, (255), thickness=cv2.FILLED)

    cv2.imshow("Finalna Cista Maska (pre razdvajanja)", final_mask)

    # 5. NOVI KORAK: Pokušaj razdvajanja spojenih objekata
    # Prosleđujemo finalnu masku i originalnu sliku
    separated_markers = separate_connected_objects(final_mask, image)

    return separated_markers

# Dimenzije i orijentacija

def analyze_and_get_object_data(markers_image, original_image, px_per_mm):
    """
    Analizira markere dobijene iz Watershed algoritma i za svaki objekat
    izračunava dimenzije, orijentaciju i centar.

    Args:
        markers_image: Slika sa labelama (markerima) iz segmentacije.
        original_image: Originalna slika, za vizuelizaciju.
        px_per_mm: Faktor kalibracije (piksela po milimetru).

    Returns:
        Lista objekata (rečnika) sa podacima i slika za prikaz.
    """
    object_data_list = []

    labels = np.unique(markers_image)
    for label in labels:
        if label <= 1:
            continue

        mask = np.zeros(markers_image.shape, dtype="uint8")
        mask[markers_image == label] = 255

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) > 0:
            c = max(contours, key=cv2.contourArea)

            if cv2.contourArea(c) < 100:
                continue

            rect = cv2.minAreaRect(c)
            (x_rect, y_rect), (width_px, height_px), angle = rect

            M = cv2.moments(c)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
            else:
                cX, cY = int(x_rect), int(y_rect)

            if px_per_mm:
                width_mm = width_px / px_per_mm
                height_mm = height_px / px_per_mm
                duzina_mm = max(width_mm, height_mm)
                sirina_mm = min(width_mm, height_mm)

                if width_px < height_px:
                    angle += 90

                object_data = {
                    "center_px": (cX, cY),
                    "center_mm": (round(cX / px_per_mm, 2), round(cY / px_per_mm, 2)),
                    "duzina_mm": round(duzina_mm, 2),
                    "sirina_mm": round(sirina_mm, 2),
                    "ugao": round(angle, 2)
                }
                object_data_list.append(object_data)

            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(original_image, [box], 0, (0, 255, 0), 2)
            cv2.circle(original_image, (cX, cY), 5, (255, 0, 0), -1)
            cv2.putText(original_image, f"ID: {label-1}", (cX, cY - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    return object_data_list, original_image

# Interfejs sa robotom

def send_data_to_robot(serial_port, object_data):
    """
    Formira string sa podacima i šalje ga preko serijskog porta.

    Args:
        serial_port: Inicijalizovan serijski port (objekat).
        object_data: Rečnik sa podacima o jednom objektu.
    """
    # Formatiranje poruke: X,Y,Duzina,Sirina,Ugao\n
    msg = (f"{object_data['center_mm'][0]},{object_data['center_mm'][1]},"
           f"{object_data['duzina_mm']},{object_data['sirina_mm']},"
           f"{object_data['ugao']}\n")

    try:
        print(f"Slanje poruke: {msg.strip()}")
        # serial_port.write(msg.encode('utf-8'))
    except Exception as e:
        print(f"Greška pri slanju podataka: {e}")


if __name__ == "__main__":
    IMAGE_DIR = "test_data_aruco"
    IMAGE_PATHS = []
    for file in os.listdir(IMAGE_DIR):
        if file.endswith('.jpg'):
            IMAGE_PATH = os.path.join(IMAGE_DIR, file)
            IMAGE_PATHS.append(IMAGE_PATH)

    ARUCO_ID_TO_FIND = 23
    ARUCO_REAL_SIZE_MM = 50

    SERIAL_PORT_NAME = 'COM3'
    BAUD_RATE = 9600

    for i, IMAGE_PATH in enumerate(IMAGE_PATHS):
        image = cv2.imread(IMAGE_PATH)
        if image is None:
            print(f"Greška: Slika na putanji '{IMAGE_PATH}' nije pronađena.")
            exit()

        px_per_mm_ratio = get_pixel_to_mm_ratio(image, ARUCO_ID_TO_FIND, ARUCO_REAL_SIZE_MM)

        image_for_segmentation = image.copy()
        segmented_markers = segment_objects(image_for_segmentation)

        output_image = image.copy()
        detected_objects, output_image_with_data = analyze_and_get_object_data(
            segmented_markers, output_image, px_per_mm_ratio
        )

        print("\n--- DETEKTOVANI OBJEKTI ---")
        if not detected_objects:
            print("Nijedan objekat nije detektovan.")
        else:
            # Inicijalizacija serijskog porta (odkomentarisati za rad sa robotom)
            # ser = None
            # try:
            #     ser = serial.Serial(SERIAL_PORT_NAME, BAUD_RATE, timeout=1)
            #     print(f"Serijski port {SERIAL_PORT_NAME} otvoren.")
            #     time.sleep(2) # Sačekati da se veza uspostavi
            # except serial.SerialException as e:
            #     print(f"Greška: Nije moguće otvoriti serijski port {SERIAL_PORT_NAME}. Radim u test modu.")
            #     print(e)

            # Slanje podataka za svaki detektovani objekat
            for i, obj in enumerate(detected_objects):
                print(f"\nObjekat #{i+1}:")
                print(f"  Centar (X, Y): {obj['center_mm']} mm")
                print(f"  Dužina: {obj['duzina_mm']} mm")
                print(f"  Širina: {obj['sirina_mm']} mm")
                print(f"  Ugao: {obj['ugao']} stepeni")

                # if ser and ser.is_open:
                #    send_data_to_robot(ser, obj)
                # else:
                #    # Ako port nije otvoren, samo ispisujemo poruku koja bi bila poslata
                send_data_to_robot(None, obj)

            # Zatvaranje serijskog porta
            # if ser and ser.is_open:
            #     ser.close()
            #     print("\nSerijski port zatvoren.")

        cv2.imwrite(f'results/output_{i}.jpg', output_image_with_data)

Marker pronađen. Odnos: 2.98 piksela/mm
Pronađeno 3 semena, pokušavam razdvajanje...

--- DETEKTOVANI OBJEKTI ---

Objekat #1:
  Centar (X, Y): (np.float32(411.41), np.float32(362.75)) mm
  Dužina: 622.72998046875 mm
  Širina: 109.98999786376953 mm
  Ugao: 130.44 stepeni
Slanje poruke: 411.4100036621094,362.75,622.72998046875,109.98999786376953,130.44

Objekat #2:
  Centar (X, Y): (np.float32(364.43), np.float32(714.77)) mm
  Dužina: 605.0999755859375 mm
  Širina: 98.20999908447266 mm
  Ugao: 143.05 stepeni
Slanje poruke: 364.42999267578125,714.77001953125,605.0999755859375,98.20999908447266,143.05

Objekat #3:
  Centar (X, Y): (np.float32(628.52), np.float32(974.5)) mm
  Dužina: 625.2100219726562 mm
  Širina: 105.80000305175781 mm
  Ugao: 15.0 stepeni
Slanje poruke: 628.52001953125,974.5,625.2100219726562,105.80000305175781,15.0
Marker pronađen. Odnos: 2.98 piksela/mm
Upozorenje: Razdvajanje nije uspelo, objekat se tretira kao jedan.

--- DETEKTOVANI OBJEKTI ---
Nijedan objekat nije d

: 